In [52]:
# Import packages.
import numpy as np

In [53]:
def generate_constraint_matrix(n, m):
    s0 = np.random.randn(m)
    s0 = np.maximum(s0, 0)
    x0 = np.random.randn(n)
    A = np.random.randn(m, n)
    b = A @ x0 + s0
    # print(b.shape)
    return A, b

In [64]:
from gekko import GEKKO
from scipy.optimize import minimize
import cvxpy as cp
from sklearn.datasets import make_spd_matrix
import time

def optimize(n, m, method):
    Q = make_spd_matrix(n)
    A, b = generate_constraint_matrix(n, m)

    start_time = time.time()

    if method == 'cvxpy':
        x = cp.Variable(n)
        prob = cp.Problem(cp.Minimize(cp.quad_form(x, Q)), [A @ x <= b])
        prob.solve()
        print("\nThe optimal value is", prob.value)
        print("A solution x is")
        print(x.value)

    elif method == 'scipy':
        x0 = np.zeros(n)
        cons = {'type': 'ineq', 'fun': lambda x: b - A @ x}
        solution = minimize(lambda x: x.T @ Q @ x, x0, constraints=cons)
        if solution.success:
            print("\nThe optimal value is", solution.fun)
            print("A solution x is")
            print(solution.x)
        else:
            print("Optimization failed with message:", solution.message)

    elif method == 'gekko':
        len = m
        m = GEKKO()
        x = m.Array(m.Var, n, lb=0)
        m.Minimize(sum(Q[i][j]*x[i]*x[j] for i in range(n) for j in range(n)))
        for i in range(len):
            m.Equation(sum(A[i][j]*x[j] for j in range(n)) <= b[i])
        m.solve(disp=False)
        print("\nThe optimal value is", m.options.objfcnval)
        print("A solution x is")
        print([xi.value[0] for xi in x])

    else:
        print("Invalid method. Choose 'cvxpy', 'scipy', or 'gekko'.")

    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"\nThe {method} method took {elapsed_time} seconds to complete.")

In [66]:
optimize(10, 5, 'cvxpy')

optimize(10, 5, 'scipy')

optimize(10, 5, 'gekko')



The optimal value is 1.9631764875087248
A solution x is
[ 0.23113209 -1.73529571  0.42628499 -0.40269036  0.46045267  0.35881995
 -0.78907096  0.15023587  0.62952488 -0.05679208]

The cvxpy method took 0.01756119728088379 seconds to complete.

The optimal value is 1.4590680345440348
A solution x is
[ 0.48663569  0.57463405  1.05819848  0.1495245   1.04820072  0.7166059
 -0.13431133 -0.33324471  0.60814438  0.32517035]

The scipy method took 0.007833480834960938 seconds to complete.

The optimal value is 5.4874898056
A solution x is
[0.0, 0.96838520026, 0.0, 0.0027048672768, 0.0, 2.4044229862, 0.77455586317, 0.27045561312, 0.16060251478, 0.0]

The gekko method took 8.440691947937012 seconds to complete.
